## Raw data collection
This script collects raw data from export from MMI Items menu into csv-files usable for
reorder point optimization.

In [20]:
import numpy as np
import pandas as pd
import math

In [22]:
raw_data_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_data_raw_19_apr.xlsx"
raw_data_df = pd.read_excel(raw_data_path,"Raw Data")
#raw_data_df.head(5)

In [23]:
ids_df = pd.read_excel(raw_data_path,"Interesting items")
ids_df.columns = ["item code"]

## Final script
Here we will iterate through the list and save everything in csv and excel-files.

In [24]:
# Initiating excel-writer
excel_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_inputs.xlsx"
writer = pd.ExcelWriter(excel_path)

for item_code in ids_df["item code"]:
    one_item_df  = raw_data_df[raw_data_df['Item code'] == item_code]
    input_df = pd.DataFrame(columns=["Installation id", "Type", "Name", "Transport time", "Q", 
                "Unit cost", "Target item fill rate", "Demand distribution",	
                "Demand mean per time unit", "Demand stdev per time unit","Demand type","Stocked"])
    input_df["Name"] = one_item_df["Warehouse name"]
    input_df["Installation id"] = one_item_df["Warehouse name"]
    input_df["Transport time"] = one_item_df["Lead time"]
    input_df["Q"] = one_item_df["Constr. opt. OQ"]
    input_df["Unit cost"] = one_item_df["Unit cost"]
    input_df["Demand mean per time unit"] = one_item_df["EOD"].to_numpy().astype("float64")/30
    input_df["Demand stdev per time unit"] = one_item_df["Std. dev."].to_numpy().astype("float64")/math.sqrt(30)
    input_df["Demand type"] = one_item_df["Demand type"]
    input_df["Stocked"] = one_item_df["Stocked"]

    type_list = []
    for name in input_df["Name"]:
        if name == "Johannesburg":
            type_list.append("RDC")
        else:
            type_list.append("Dealer")
    input_df["Type"] = type_list

    target_IFR_list = []
    for target_rate in one_item_df["Target srv. lvl."]:
        if target_rate is not np.nan:
            target_fill_rate = (1/100)*float(target_rate[0:4])
        else:
            target_fill_rate = 0
        target_IFR_list.append(target_fill_rate)
    input_df["Target item fill rate"] = target_IFR_list
            
    input_df["Demand distribution"] = "Empiric_Compound_Poisson"

    input_df = input_df.loc[input_df["Demand type"] != "Non-moving"]
    input_df = input_df.loc[input_df["Demand type"] != "Insufficient history"]
    input_df = input_df.loc[input_df["Demand type"] != "New"]
    input_df = input_df.loc[input_df["Demand type"] != "Obsolete"]
    
    #Fixing indexes
    input_df.index = range(0,len(input_df["Installation id"]))

    # Saving a csv
    google_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_input_csv_files"
    csv_path = f"{google_path}/item_{item_code}_input.csv"
    input_df.to_csv(csv_path)

    # Writing to excel
    excel_sheet_name = f"item_{item_code}_input"
    input_df.to_excel(writer, sheet_name = excel_sheet_name) 

writer.save()